# Oracle RAG Agents: Zero to Hero

[![Open in Colab](https://img.shields.io/badge/Open%20in-Colab-F9AB00?style=flat-square&logo=googlecolab)](https://colab.research.google.com/github/oracle-devrel/oracle-ai-developer-hub/blob/main/notebooks/oracle_rag_agents_zero_to_hero.ipynb)

-------


## What You Will Learn

In this notebook, you will learn how to:
- Set up Oracle AI Database locally for AI application development.
- Build a research dataset pipeline from ingestion to embeddings.
- Implement keyword, vector, hybrid, and graph-based retrieval in Oracle.
- Create a full RAG workflow and connect it to OpenAI models.
- Build agentic workflows with tool use, orchestration, and session memory.


In [ ]:
! pip install -Uq oracledb pandas sentence-transformers datasets einops "numpy<2.0"

# Part 1. Oracle AI Database Local Installation

## 1.1 Local Installation of Oracle AI Database

**Oracle AI Database 26ai** is a **converged database** built for AI developers.  
It unifies **relational, document, graph and vector data** in one engine — so you can build  
**semantic search**, **RAG**, and **natural language to SQL** applications without leaving the database.  

Store embeddings, run vector search, and apply AI directly where your data lives —  
**securely, efficiently, and at scale.**


For this notebook we will be using a local installation of [Oracle AI Database](https://www.oracle.com/database/free/get-started/)

1. Install & start Docker. Docker Desktop (Mac/Windows) or Docker Engine (Linux). Make sure it’s running.
    - If installed with Docker Enginer, run from terminal ```open /Applications/Docker.app```
2. Pull [docker image](https://www.oracle.com/database/free/get-started/)
3. Run a container with oracle image

    ```
    docker run -d \
      --name oracle-full \
      -p 1521:1521 -p 5500:5500 \
      -e ORACLE_PWD=YourStrongPassword \
      -e ORACLE_SID=FREE \
      -e ORACLE_PDB=FREEPDB1 \
      -v ~/oracle/full_data:/opt/oracle/oradata \
      container-registry.oracle.com/database/free:latest
    ```

> 🚫 **Troubleshoot**  
> If you see the error:  
> *`docker: Error response from daemon: Conflict. The container name "/oracle-full" is already in use by container ... You have to remove (or rename) that container to be able to reuse that name.`*  
>
> 🧩 **Fix:**  
> - Remove the existing container:  
>   ```bash
>   docker rm oracle-full
>   ```  
> - Then re-run your Docker command from **Step 3** to start a new container.


After running the docker command above in your terminal, you should see the image below if you click into the container running

### 1.1.1 Connecting to Oracle AI Database

In [ ]:
import os
import oracledb
import time


def ensure_property_graph_privileges(
    target_user: str = "VECTOR",
    admin_user: str = "SYSTEM",
    admin_password: str | None = None,
    dsn: str = "localhost:1521/FREEPDB1",
):
    """
    Grant SQL Property Graph privileges required by this notebook.

    Uses an admin session (SYSTEM by default) to grant:
      - CREATE PROPERTY GRAPH (required)
      - READ ANY PROPERTY GRAPH (optional; useful for cross-schema graph reads)

    Set ORACLE_ADMIN_PWD in your environment if admin password differs.
    """
    admin_password = (
        admin_password
        or os.environ.get("ORACLE_ADMIN_PWD")
        or os.environ.get("ORACLE_PASSWORD")
        or "YourStrongPassword"
    )

    try:
        with oracledb.connect(user=admin_user, password=admin_password, dsn=dsn) as admin_conn:
            with admin_conn.cursor() as cur:
                # Required for CREATE PROPERTY GRAPH DDL
                cur.execute(f"GRANT CREATE PROPERTY GRAPH TO {target_user}")

                # Optional in some setups/versions; ignore if unsupported
                try:
                    cur.execute(f"GRANT READ ANY PROPERTY GRAPH TO {target_user}")
                    print(f"✓ Granted READ ANY PROPERTY GRAPH to {target_user}.")
                except oracledb.DatabaseError as read_err:
                    if "ORA-00990" in str(read_err):
                        print("ℹ️ READ ANY PROPERTY GRAPH is not available in this setup; continuing.")
                    else:
                        raise

            admin_conn.commit()

        print(f"✓ Granted CREATE PROPERTY GRAPH to {target_user}.")

    except oracledb.DatabaseError as e:
        print(f"⚠️ Could not auto-grant property graph privileges as {admin_user}: {e}")
        print("   If you hit ORA-01031 later, run as admin:")
        print(f"   GRANT CREATE PROPERTY GRAPH TO {target_user};")
        print(f"   -- Optional if supported: GRANT READ ANY PROPERTY GRAPH TO {target_user};")


def connect_to_oracle(max_retries=3, retry_delay=5):
    """
    Connect to Oracle database with retry logic and better error handling.

    Args:
        max_retries: Maximum number of connection attempts
        retry_delay: Seconds to wait between retries
    """
    user = "VECTOR"
    password = "VectorPwd_2025"  # must match ORACLE_PWD from docker run
    dsn = "localhost:1521/FREEPDB1"

    for attempt in range(1, max_retries + 1):
        try:
            print(f"Connection attempt {attempt}/{max_retries}...")
            conn = oracledb.connect(
                user=user,
                password=password,
                dsn=dsn
            )
            print("✓ Connected successfully!")

            # Test the connection
            with conn.cursor() as cur:
                cur.execute("SELECT banner FROM v$version WHERE banner LIKE 'Oracle%';")
                banner = cur.fetchone()[0]
                print(f"\n{banner}")

            return conn

        except oracledb.OperationalError as e:
            error_msg = str(e)
            print(f"✗ Connection failed (attempt {attempt}/{max_retries})")

            if "DPY-4011" in error_msg or "Connection reset by peer" in error_msg:
                print("  → This usually means:")
                print("    1. Database is still starting up (wait 2-3 minutes)")
                print("    2. Listener is not bound to 0.0.0.0 (run fix_oracle_listener())")
                print("    3. Container is not running (check with check_docker_container())")

                if attempt < max_retries:
                    print(f"\n  Waiting {retry_delay} seconds before retry...")
                    time.sleep(retry_delay)
                else:
                    print("\n  💡 Try running:")
                    print("     1. check_docker_container() - verify container is running")
                    print("     2. fix_oracle_listener() - fix listener binding")
                    raise
            else:
                raise
        except Exception as e:
            print(f"✗ Unexpected error: {e}")
            raise

    raise ConnectionError("Failed to connect after all retries")


# Ensure VECTOR can create/read SQL Property Graph objects
ensure_property_graph_privileges(
    target_user="VECTOR",
    admin_user="SYSTEM",
    dsn="localhost:1521/FREEPDB1",
)

# Connect to Oracle as VECTOR
conn = connect_to_oracle()


> 🚫 Troubleshoot: Oracle Database Free (Docker) — Connection Fix
>
> If you see errors like:
>
> ```
> OperationalError: DPY-6005: cannot connect to database
> DPY-4011: the database or network closed the connection
> TNS-12545: Connect failed because target host or object does not exist
> ```
>
> It means the **Oracle listener** inside the container is binding to the **container hostname** instead of `0.0.0.0`, preventing host connections.
>
>
>
> 🧩 Fix
> 
> Run this exact command to patch the listener and restart it:
> 
> ```bash
> docker exec -it oracle-full bash -lc '
>   export ORACLE_HOME=${ORACLE_HOME:-/opt/oracle/product/26ai/dbhomeFree}
>   export PATH=$ORACLE_HOME/bin:$PATH
>   LISTENER_ORA="$ORACLE_HOME/network/admin/listener.ora"
> 
>   echo "== Fixing listener to use HOST=0.0.0.0"
>   sed -i "s/(HOST *= *[^)]*)/(HOST = 0.0.0.0)/" "$LISTENER_ORA"
> 
>   echo "== Restarting listener"
>   lsnrctl stop || true
>   lsnrctl start
> 
>   echo "== Re-registering services"
>   echo "ALTER SYSTEM REGISTER;" | sqlplus -s / as sysdba
> 
>   echo "== Listener status (first 20 lines):"
>   lsnrctl status | sed -n "1,20p"
> '
> ```
> 
> This:
> - Forces the listener to bind on all interfaces (`0.0.0.0`).
> - Restarts the listener.
> - Re-registers the PDB service (`FREEPDB1`) with the listener.
> 
> ---
> 


## 1.2 Remote Access with FreeSQL.com (Coming Soon)

---------

# Part 2. Data Loading, Preparation and Embedding Generation

## 2.1 Data Loading From Hugging Face

**Streaming the ArXiv Papers Dataset with Hugging Face**

The following code in the next cell demonstrates how to efficiently load and stream a large dataset using the **Hugging Face `datasets`** library — a powerful tool for handling massive datasets that may not fit into memory all at once.

```python
import pandas as pd
from datasets import load_dataset

ds_stream = load_dataset("nick007x/arxiv-papers", split="train", streaming=True)
```

**Step-by-Step Explanation**
1. **Importing dependencies**
   - `load_dataset` is imported from the `datasets` library, giving access to thousands of open datasets hosted on the Hugging Face Hub.

2. **Loading the dataset**
   - The dataset `"nick007x/arxiv-papers"` refers to a public dataset on the Hugging Face Hub that contains metadata or text from research papers hosted on [arXiv.org](https://arxiv.org).
   - The parameter `split="train"` loads the training partition of the dataset (many datasets have `train`, `validation`, and `test` splits).
   - The key argument `streaming=True` activates **streaming mode**, meaning the dataset is read progressively from the source without downloading it entirely to disk.

3. **Why streaming mode matters**
   - Traditional dataset loading downloads the full dataset into memory or disk, which can be slow and memory-intensive.  
   - Streaming allows you to process examples **as they arrive**, ideal for very large datasets or limited-resource environments.
   - You can iterate over the dataset like this:
     ```python
     for record in ds_stream:
         print(record)
         break
     ```

4. **Resulting object**
   - The variable `ds_stream` is an instance of `datasets.IterableDataset`, not a static table.  
   - You can convert a small sample into a Pandas DataFrame for inspection:
     ```python
     sample = [next(iter(ds_stream)) for _ in range(5)]
     pd.DataFrame(sample)
     ```

> **💡 Takeaway:**  
> Using `load_dataset(..., streaming=True)` enables developers and data scientists to work with **large datasets efficiently** — a perfect fit for machine learning pipelines, LLM training, or large document analysis workflows.


In [ ]:
import pandas as pd
from datasets import load_dataset

ds_stream = load_dataset("nick007x/arxiv-papers", split="train", streaming=True)

The code below streams the first 1,000 ArXiv papers(feel free to use more) from the dataset, extracts their titles and abstracts, combines them into a single text field, and stores the results as structured dictionaries for later use.

In [ ]:
sampled = []
for i, item in enumerate(ds_stream):
    if i >= 1000:
        break
    
    arxiv_id = item.get("arxiv_id", f"unknown_{i}")
    title = item.get("title", "").strip()
    abstract = item.get("abstract", "").strip()
    authors = item.get("authors", [])

    if isinstance(authors, str):
        authors = [a.strip() for a in authors.split(",") if a.strip()]
    elif isinstance(authors, list):
        authors = [str(a).strip() for a in authors if str(a).strip()]
    else:
        authors = []
    
    # Combine title + abstract for embedding text
    text = f"{title} — {abstract}"
    
    sampled.append({
        "id": item.get("id", f"arxiv_{i}"),
        "arxiv_id": arxiv_id,
        "title": title,
        "abstract": abstract,
        "authors": authors,
        "text": text
    })

print(f"✅ Streamed {len(sampled)} papers.")


The code below converts the collected list of sampled paper records into a Pandas DataFrame for easier analysis, prints how many rows were loaded, and displays the first few entries to preview the dataset’s structure.

In [ ]:
# Convert the list of tuples (id, text) into a DataFrame
dataset_df = pd.DataFrame(sampled)

# View shape and head
print(f"✅ Loaded {len(dataset_df)} rows into DataFrame.")
dataset_df.head()

## 2.2 Embedding Generation


This code in the next cell below imports the **`SentenceTransformer`** class from the `sentence_transformers` library and loads a pretrained model called **`"nomic-ai/nomic-embed-text-v1.5"`** from the Hugging Face Hub.

```python
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)
```

🔍 What it does
- **`SentenceTransformer`** extends transformer-based models (like BERT or RoBERTa) to produce **sentence-level embeddings** — dense numerical vectors that capture the semantic meaning of text.
- The model **`nomic-ai/nomic-embed-text-v1.5`** is optimized for general-purpose text embeddings and works well for tasks such as:
  - Semantic search  
  - Clustering and topic modeling  
  - Retrieval-Augmented Generation (RAG)  
  - Similarity ranking and recommendation systems
- The parameter **`trust_remote_code=True`** allows the loader to execute custom code from the model’s repository, which is required for models that define specialized architectures or preprocessing logic.

In short, this code prepares a powerful embedding model that can transform text (like paper titles or abstracts) into **high-dimensional semantic vectors**, making it easier to measure meaning-based similarity across documents.


In [ ]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("nomic-ai/nomic-embed-text-v1.5", trust_remote_code=True)

This code in the next cell below iterates through each document, encodes it into a normalized embedding vector while showing live progress, and prefixes each text with "search_document: " so that the Nomic embedding model correctly interprets it as a retrieval document, improving alignment with query embeddings during semantic search.

The prefix `search_document`: " tells the Nomic embedding model what kind of text it’s encoding — in this case, a document intended for retrieval.

Nomic models like nomic-embed-text-v1.5 are trained with instructional prefixes (e.g., "search_query:", "search_document:", "classification:"), which guide the model to generate embeddings suited for different purposes.

**Why Use the `"search_document:"` Prefix with Nomic Embeddings**

Nomic embedding models (like **`nomic-embed-text-v1.5`**) are **instruction-tuned**, meaning they were trained with specific **task prefixes** that tell the model *how* to interpret the text you’re embedding.  

According to the [Nomic documentation](https://docs.nomic.ai/reference/api/embed-text-v-1-embedding-text-post) and the [Hugging Face model card](https://huggingface.co/nomic-ai/nomic-embed-text-v1.5):

> “Important: the text prompt must include a task instruction prefix, instructing the model which task is being performed.
> For example, if you are implementing a RAG application, you embed your documents as  
> `search_document: <text>` and embed your user queries as `search_query: <text>`.”

**💡 Why this matters**
- The prefix tells the model whether a text is a **document** or a **query**, ensuring both are embedded into the **same semantic space**.
- Using `search_document:` for document embeddings and `search_query:` for query embeddings **improves retrieval accuracy**, since the model optimizes for similarity between matching query–document pairs.
- Omitting or mismatching prefixes can lead to weaker alignment and lower recall in search or RAG workflows.


In [ ]:
import numpy as np
import sys

# Prefix for retrieval-style embeddings
dataset_df["text_prefixed"] = dataset_df["text"].apply(lambda x: f"search_document: {x}")

# Convert to list for iteration
texts = dataset_df["text_prefixed"].tolist()
embs = []

# Encode one text at a time with a single-line progress display
total = len(texts)
for i, text in enumerate(texts, start=1):
    embedding = embedding_model.encode(
        text,
        show_progress_bar=False,   # Disable SentenceTransformer progress
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    embs.append(embedding)

    # Print progress on the same line
    sys.stdout.write(f"\rEncoding {i}/{total} texts...")
    sys.stdout.flush()

# Final newline to avoid overwriting the last line
print(f"\n✅ Finished encoding {len(embs)} texts.")

# Convert list of vectors to NumPy array
embs = np.vstack(embs)
print(f"✅ Embeddings shape: {embs.shape}")


One important detail to note is the embedding dimensionality — the number of numerical features in each vector representation.
This dimensionality determines the structure of your vector index and must remain consistent across all embeddings to ensure efficient similarity search and accurate retrieval performance.

This code in the next cell below converts each embedding into a list of 32-bit floating-point numbers (float32) so it can be stored in an Oracle VECTOR column, determines the embedding dimension (needed for defining the vector index), and we then displays the first two rows to confirm the data and embeddings were formatted correctly.

In [ ]:
# store as float32 lists (ready for Oracle VECTOR column)
dataset_df["embedding"] = [e.astype(np.float32).tolist() for e in embs]

dim = len(dataset_df["embedding"].iloc[0])

# View the first 2 rows of the dataset
dataset_df.head(2)

----

# Part 3: Database Table Setup and Data Ingestion

This code below safely resets the core paper table by first dropping any dependent graph edge tables (if they exist), then recreating `research_papers` with paper metadata and a `VECTOR` embedding column of size `dim`.

This rerun-safe order avoids foreign-key drop errors when graph tables already reference `research_papers`.

## 3.1 Create Reseach Papers Table

In [ ]:
ddl = f"""
BEGIN
    EXECUTE IMMEDIATE 'DROP TABLE paper_similarities';
EXCEPTION WHEN OTHERS THEN
    IF SQLCODE != -942 THEN RAISE; END IF;
END;
/
BEGIN
    EXECUTE IMMEDIATE 'DROP TABLE paper_authors';
EXCEPTION WHEN OTHERS THEN
    IF SQLCODE != -942 THEN RAISE; END IF;
END;
/
BEGIN
    EXECUTE IMMEDIATE 'DROP TABLE authors';
EXCEPTION WHEN OTHERS THEN
    IF SQLCODE != -942 THEN RAISE; END IF;
END;
/
BEGIN
    EXECUTE IMMEDIATE 'DROP TABLE research_papers CASCADE CONSTRAINTS PURGE';
EXCEPTION WHEN OTHERS THEN
    IF SQLCODE != -942 THEN RAISE; END IF;
END;
/
CREATE TABLE research_papers (
    arxiv_id VARCHAR2(255) PRIMARY KEY,
    title VARCHAR2(4000),
    abstract VARCHAR2(4000),
    text CLOB,
    embedding VECTOR({dim}, FLOAT32)
)
TABLESPACE USERS
"""


Here, we’re taking the multi-statement SQL stored in `ddl`, splitting it by `/` so each command runs separately, and executing them one by one using the database cursor.  
After all statements finish, we call `conn.commit()` to save the changes — effectively creating the `RESEARCH_PAPERS` table — and then print a confirmation message showing the vector dimension (`dim`) used for the `embedding` column.


In [ ]:
with conn.cursor() as cur:
    for stmt in ddl.split("/"):
        if stmt.strip():
            cur.execute(stmt)

conn.commit()
print("✅ Table RESEARCH_PAPERS created with VECTOR dimension:", dim)

## 3.2 Create Indexes (Vector and Search)

This code in the next cell below creates a **vector index** on the `embedding` column of the `research_papers` table to enable fast similarity search.  
The index, named `RP_VEC_HNSW`, uses Oracle’s `VECTOR` indexing with **HNSW (Hierarchical Navigable Small World)** organization, which stores vectors in a graph for efficient approximate nearest-neighbor traversal.  
It’s configured to use **cosine distance** as the similarity metric and a **target accuracy of 90%**, balancing search speed and precision.  
We also set HNSW build parameters (`NEIGHBORS`, `EFCONSTRUCTION`) for graph connectivity and index quality.  
Finally, `conn.commit()` saves the index creation, and a confirmation message is printed once the index is successfully built.

Note: In Oracle’s vector indexing syntax, **ORGANIZATION INMEMORY NEIGHBOR GRAPH** enables the HNSW graph-based index structure.


In [ ]:
with conn.cursor() as cur:
    # Drop old vector indexes if they exist (supports re-running this section)
    for idx_name in ("RP_VEC_HNSW", "RP_VEC_IVF"):
        try:
            cur.execute(f"DROP INDEX {idx_name}")
        except oracledb.DatabaseError as e:
            if "ORA-01418" not in str(e):  # ignore "index does not exist"
                raise

    cur.execute("""
        CREATE VECTOR INDEX RP_VEC_HNSW
        ON research_papers(embedding)
        ORGANIZATION INMEMORY NEIGHBOR GRAPH
        DISTANCE COSINE
        WITH TARGET ACCURACY 90
        PARAMETERS (TYPE HNSW, NEIGHBORS 40, EFCONSTRUCTION 500)
        TABLESPACE USERS
    """)

conn.commit()
print("✅ Vector Index RP_VEC_HNSW (HNSW) created")


> **💡 Knowledge Checkpoint**
>
> HNSW (Hierarchical Navigable Small World) is a graph-based vector index that speeds up similarity search using layered small-world navigation.
>
> Instead of scanning all vectors, HNSW traverses graph links from coarse to fine levels to quickly approach nearest neighbors.
> 
> In Oracle AI Database, the clause `ORGANIZATION INMEMORY NEIGHBOR GRAPH` activates this HNSW structure. Parameters like `NEIGHBORS` and `EFCONSTRUCTION` control graph connectivity and index build quality, while `WITH TARGET ACCURACY` controls ANN search quality.


**Creating an Oracle Text Index for Full-Text Search**

This code below creates a **full-text search index** on the `text` column of the `research_papers` table using Oracle Text.

1. **Drop existing index** — removes `rp_text_idx` if it already exists, ignoring the “index does not exist” error.  
2. **Create new text index** — builds a `CTXSYS.CONTEXT` index, which tokenizes and indexes the text for efficient keyword searches.  
   - `SYNC (ON COMMIT)` keeps the index automatically updated whenever new data is committed.  
3. **Commit and confirm** — saves the changes and prints a success message.

Once created, you can use the `CONTAINS()` operator with `SCORE()` for fast, ranked keyword searches, e.g.:

```sql
SELECT title, SCORE(1) AS relevance
FROM research_papers
WHERE CONTAINS(text, 'transformer architecture', 1) > 0
ORDER BY SCORE(1) DESC;
```

This turns your text column into a search-optimized field, similar to how search engines handle full-text retrieval.


In [ ]:
with conn.cursor() as cur:
    # Drop old index if it exists
    try:
        cur.execute("DROP INDEX rp_text_idx")
    except oracledb.DatabaseError as e:
        if "ORA-01418" not in str(e):  # ignore "index does not exist"
            raise

    # Create a TEXT index on the 'text' column only
    cur.execute("""
        CREATE INDEX rp_text_idx
        ON research_papers(text)
        INDEXTYPE IS CTXSYS.CONTEXT
        PARAMETERS ('SYNC (ON COMMIT)')
    """)

conn.commit()
print("✅ Oracle Text index (rp_text_idx) created successfully on TEXT column.")


### 3.2.1 Create Relational Tables for Graph Retrieval

To support Oracle SQL Property Graph retrieval, we add normalized edge tables for authorship and paper similarity. This keeps the existing `research_papers` vector/text setup intact while adding graph traversal data.


In [ ]:
graph_tables_ddl = """
BEGIN
    EXECUTE IMMEDIATE 'DROP TABLE paper_similarities';
EXCEPTION WHEN OTHERS THEN
    IF SQLCODE != -942 THEN RAISE; END IF;
END;
/
BEGIN
    EXECUTE IMMEDIATE 'DROP TABLE paper_authors';
EXCEPTION WHEN OTHERS THEN
    IF SQLCODE != -942 THEN RAISE; END IF;
END;
/
BEGIN
    EXECUTE IMMEDIATE 'DROP TABLE authors';
EXCEPTION WHEN OTHERS THEN
    IF SQLCODE != -942 THEN RAISE; END IF;
END;
/
CREATE TABLE authors (
    author_name VARCHAR2(512) PRIMARY KEY
)
TABLESPACE USERS
/
CREATE TABLE paper_authors (
    arxiv_id VARCHAR2(255) NOT NULL,
    author_name VARCHAR2(512) NOT NULL,
    CONSTRAINT pk_paper_authors PRIMARY KEY (arxiv_id, author_name),
    CONSTRAINT fk_pa_paper FOREIGN KEY (arxiv_id) REFERENCES research_papers(arxiv_id),
    CONSTRAINT fk_pa_author FOREIGN KEY (author_name) REFERENCES authors(author_name)
)
TABLESPACE USERS
/
CREATE TABLE paper_similarities (
    source_arxiv_id VARCHAR2(255) NOT NULL,
    target_arxiv_id VARCHAR2(255) NOT NULL,
    sim_score NUMBER(8,6) NOT NULL,
    rank_no NUMBER(5) NOT NULL,
    CONSTRAINT pk_paper_similarities PRIMARY KEY (source_arxiv_id, target_arxiv_id),
    CONSTRAINT fk_ps_src FOREIGN KEY (source_arxiv_id) REFERENCES research_papers(arxiv_id),
    CONSTRAINT fk_ps_tgt FOREIGN KEY (target_arxiv_id) REFERENCES research_papers(arxiv_id),
    CONSTRAINT ck_ps_not_self CHECK (source_arxiv_id <> target_arxiv_id)
)
TABLESPACE USERS
"""

with conn.cursor() as cur:
    for stmt in graph_tables_ddl.split("/"):
        if stmt.strip():
            cur.execute(stmt)

    cur.execute("CREATE INDEX idx_pa_author ON paper_authors(author_name)")
    cur.execute("CREATE INDEX idx_ps_source ON paper_similarities(source_arxiv_id)")
    cur.execute("CREATE INDEX idx_ps_target ON paper_similarities(target_arxiv_id)")

conn.commit()
print("✅ Graph tables created: AUTHORS, PAPER_AUTHORS, PAPER_SIMILARITIES")


In the code below we’re taking all the research paper records (including their embeddings) from the DataFrame and inserting them into the `RESEARCH_PAPERS` table in Oracle.

Let’s break down what’s happening:


1. Convert each embedding for Oracle compatibility
```python
embedding_array = array.array('f', row.get("embedding"))
```
Oracle’s `VECTOR` column expects the data as a compact binary float array, not a Python list.  
So here we convert each embedding into an `array.array('f')` — this ensures the data binds correctly and efficiently when inserting.


2. Prepare all rows for insertion
For every paper, we build a tuple containing its metadata (`arxiv_id`, `title`, `abstract`, `text`) and the formatted `embedding_array`.  
These tuples are collected in a list called `rows`.

3. Insert each row with a progress bar
```python
for row in tqdm(rows):
    cur.execute("""
        INSERT INTO research_papers (arxiv_id, title, abstract, text, embedding)
        VALUES (:1, :2, :3, :4, :5)
    """, row)
```
We loop through each row, inserting it into the table using Oracle’s parameterized query syntax.  
The `tqdm` progress bar gives real-time feedback on the insertion process — helpful when inserting hundreds or thousands of embeddings.


4. Commit everything
Finally, `conn.commit()` saves all the inserted records permanently in the database.  You’ll see a confirmation message once the operation completes successfully.


## 3.3 Ingest Data into Oracle

In [ ]:
from tqdm import tqdm
import array


rows = []
for i, row in dataset_df.iterrows():
    # Convert embedding list to array.array for proper VECTOR binding
    embedding_array = array.array('f', row.get("embedding"))
    
    rows.append((
        row.get("arxiv_id"),
        row.get("title"),
        row.get("abstract"),
        row.get("text"),
        embedding_array
    ))

print(f"Preparing to insert {len(rows)} rows into RESEARCH_PAPERS...")

with conn.cursor() as cur:
    for row in tqdm(rows):
        cur.execute(
            """
            INSERT INTO research_papers (arxiv_id, title, abstract, text, embedding)
            VALUES (:1, :2, :3, :4, :5)
            """, 
            row
        )
        
conn.commit()
print("✅ Data inserted successfully")

In [ ]:
with conn.cursor() as cur:
    cur.execute("SELECT COUNT(*) FROM RESEARCH_PAPERS")
    print("Row count:", cur.fetchone()[0])

    cur.execute("""
        SELECT arxiv_id, title, abstract, text FROM RESEARCH_PAPERS
        FETCH FIRST 3 ROWS ONLY
    """)
    for row in cur.fetchall():
        print(row)

## 3.4 Build and Register Graph Relationships


### 3.4.1 Load Author Edges (`Author -> WROTE -> Paper`)

Here we normalize the dataset `authors` field and populate `AUTHORS` + `PAPER_AUTHORS` so graph traversal can connect papers through shared authors.


In [ ]:
def normalize_authors(raw_authors):
    if raw_authors is None:
        return []
    if isinstance(raw_authors, str):
        return [a.strip() for a in raw_authors.split(",") if a.strip()]
    if isinstance(raw_authors, list):
        cleaned = []
        for author in raw_authors:
            text = str(author).strip()
            if text:
                cleaned.append(text)
        return cleaned
    return []

author_rows = set()
paper_author_rows = set()

for _, row in dataset_df.iterrows():
    arxiv_id = row.get("arxiv_id")
    if not arxiv_id:
        continue

    for author_name in normalize_authors(row.get("authors")):
        author_rows.add((author_name,))
        paper_author_rows.add((arxiv_id, author_name))

with conn.cursor() as cur:
    cur.execute("TRUNCATE TABLE paper_authors")
    cur.execute("TRUNCATE TABLE authors")

    if author_rows:
        cur.executemany(
            "INSERT INTO authors (author_name) VALUES (:1)",
            sorted(author_rows)
        )

    if paper_author_rows:
        cur.executemany(
            "INSERT INTO paper_authors (arxiv_id, author_name) VALUES (:1, :2)",
            sorted(paper_author_rows)
        )

conn.commit()
print(f"✅ Loaded {len(author_rows)} authors and {len(paper_author_rows)} Author->WROTE->Paper edges.")


### 3.4.2 Load Similarity Edges (`Paper -> SIMILAR_TO -> Paper`)

Using the normalized embeddings already computed for vector search, we add top-k nearest-neighbor paper links and persist them as directed similarity edges.


In [ ]:
TOP_SIM_NEIGHBORS = 10

paper_ids = dataset_df["arxiv_id"].tolist()
emb_matrix = np.vstack(dataset_df["embedding"].values).astype(np.float32)

neighbor_k = min(TOP_SIM_NEIGHBORS, max(len(paper_ids) - 1, 0))
similarity_rows = []

if neighbor_k > 0:
    similarity_matrix = emb_matrix @ emb_matrix.T
    np.fill_diagonal(similarity_matrix, -np.inf)

    for i, source_arxiv_id in enumerate(paper_ids):
        nn_idx = np.argpartition(similarity_matrix[i], -neighbor_k)[-neighbor_k:]
        nn_idx = nn_idx[np.argsort(similarity_matrix[i][nn_idx])[::-1]]

        for rank_no, j in enumerate(nn_idx, start=1):
            similarity_rows.append((
                source_arxiv_id,
                paper_ids[j],
                float(similarity_matrix[i][j]),
                rank_no
            ))

with conn.cursor() as cur:
    cur.execute("TRUNCATE TABLE paper_similarities")
    if similarity_rows:
        cur.executemany(
            """
            INSERT INTO paper_similarities (source_arxiv_id, target_arxiv_id, sim_score, rank_no)
            VALUES (:1, :2, :3, :4)
            """,
            similarity_rows
        )

conn.commit()
print(f"✅ Loaded {len(similarity_rows)} Paper->SIMILAR_TO->Paper edges (top {neighbor_k} per paper).")


### 3.4.3 Register the Oracle SQL Property Graph

This cell creates a SQL Property Graph (`RESEARCH_GRAPH`) over `RESEARCH_PAPERS`, `AUTHORS`, `PAPER_AUTHORS`, and `PAPER_SIMILARITIES`, so we can use `GRAPH_TABLE` pattern matching for graph-native retrieval.


In [ ]:
with conn.cursor() as cur:
    cur.execute("""
        SELECT COUNT(*)
        FROM user_property_graphs
        WHERE graph_name = 'RESEARCH_GRAPH'
    """)
    graph_exists = cur.fetchone()[0] > 0

    if graph_exists:
        cur.execute("DROP PROPERTY GRAPH research_graph")

    cur.execute("""
        CREATE PROPERTY GRAPH research_graph
        VERTEX TABLES (
            research_papers
                KEY (arxiv_id)
                LABEL paper
                PROPERTIES (arxiv_id, title, abstract),
            authors
                KEY (author_name)
                LABEL author
                PROPERTIES (author_name)
        )
        EDGE TABLES (
            paper_authors
                KEY (arxiv_id, author_name)
                SOURCE KEY (author_name) REFERENCES authors (author_name)
                DESTINATION KEY (arxiv_id) REFERENCES research_papers (arxiv_id)
                LABEL wrote,
            paper_similarities
                KEY (source_arxiv_id, target_arxiv_id)
                SOURCE KEY (source_arxiv_id) REFERENCES research_papers (arxiv_id)
                DESTINATION KEY (target_arxiv_id) REFERENCES research_papers (arxiv_id)
                LABEL similar_to
                PROPERTIES (sim_score, rank_no)
        )
    """)

conn.commit()
print("✅ SQL Property Graph RESEARCH_GRAPH created.")


# Part 4. Retrieval Mechanisms

In [ ]:
SEARCH_TEXT_KEYWORDS = "optimization"

## 4.1 Text Based Retrieval

The code below performs a full-text search over the text column of the research_papers table, using the Oracle Text index we just created earlier

- CONTAINS(text, :keyword, 1) uses the Oracle Text index on the text column to find documents containing the given keyword or phrase.
- SCORE(1) assigns a relevance score based on how well each document matches the search term.
- SUBSTR(text, 1, 200) returns the first 200 characters as a short preview snippet.
- FETCH FIRST 10 ROWS ONLY limits the results to the top 10 most relevant matches.

In [ ]:
def keyword_search_research_papers(conn, keyword: str):
    """
    Perform a full-text keyword search on the 'text' column 
    using the Oracle Text index (rp_text_idx).

    Args:
        conn: Oracle database connection object.
        keyword (str): Keyword or phrase to search for.

    Returns:
        tuple: (rows, columns)
    """
    query = """
        SELECT 
            arxiv_id, 
            title, 
            SUBSTR(text, 1, 200) AS text_snippet,
            SCORE(1) AS relevance_score
        FROM research_papers
        WHERE CONTAINS(text, :keyword, 1) > 0
        ORDER BY SCORE(1) DESC
        FETCH FIRST 10 ROWS ONLY
    """

    with conn.cursor() as cur:
        cur.execute(query, keyword=keyword)
        rows = cur.fetchall()
        columns = [desc[0] for desc in cur.description]

    return rows, columns


In [ ]:
rows, columns = keyword_search_research_papers(conn, SEARCH_TEXT_KEYWORDS)

results_df = pd.DataFrame(rows, columns=columns)

print(f"🔍 Keyword Search: '{SEARCH_TEXT_KEYWORDS}'")
print(f"📊 Results: {len(results_df)}\n")

results_df


As shown above, the returned rows represent documents whose indexed `text` content matched the full-text search query **"Optimization"**, as determined by the Oracle Text `CONTAINS()` operator using the `rp_text_idx` index.


## 4.2 Vector Based Retrieval

In [ ]:
SEARCH_QUERY = "Get me papers related to planetary exploration"

**Vector Similarity Search in Oracle**

This function below `vector_search_research_papers` performs a **semantic vector search** on the `research_papers` table, retrieving papers most similar to a given query using **Oracle’s native VECTOR search** feature.


Step-by-step overview

1. Encode the search query
```python
query_embedding = embedding_model.encode(
    [f"search_query: {search_query}"],
    convert_to_numpy=True,
    normalize_embeddings=True
)[0].astype(np.float32).tolist()
```
- The query is embedded using the same model that generated the document embeddings.  
- The prefix `"search_query:"` ensures embeddings align with `"search_document:"` vectors.  
- Normalized and cast to `float32` for Oracle’s `VECTOR` type.


2. Prepare for database binding
```python
query_embedding_array = array.array('f', query_embedding)
```
- Converts the embedding list into a binary float array (`array('f')`), required for Oracle’s vector binding.


3. Perform vector search
```sql
SELECT arxiv_id, title, abstract,
       SUBSTR(text, 1, 200) AS text_snippet,
        ROUND(1 - VECTOR_DISTANCE(embedding, :q, COSINE), 4) AS similarity_score
FROM research_papers
ORDER BY similarity_score
FETCH APPROX FIRST {top_k} ROWS ONLY WITH TARGET ACCURACY 90
```
- Computes **cosine distance** between the query vector and stored embeddings.  
- Orders results by similarity, returning the closest matches.  
- Uses **Approximate Nearest Neighbor (ANN)** search for fast retrieval at 90% target accuracy.

In [ ]:
def vector_search_research_papers(conn, embedding_model, search_query: str, top_k: int = 5):
    """
    Perform a vector similarity search on the research_papers table using a query embedding.
    Returns cosine similarity scores (higher = more similar).
    """

    # 1️⃣ Encode the query into a vector
    query_embedding = embedding_model.encode(
        [f"search_query: {search_query}"], 
        convert_to_numpy=True,
        normalize_embeddings=True
    )[0].astype(np.float32).tolist()

    # 2️⃣ Prepare the vector for Oracle binding
    query_embedding_array = array.array('f', query_embedding)

    # 3️⃣ Run a vector similarity search using cosine similarity
    query = f"""
        SELECT 
            arxiv_id, 
            title, 
            abstract, 
            SUBSTR(text, 1, 200) AS text_snippet,
            ROUND(1 - VECTOR_DISTANCE(embedding, :q, COSINE), 4) AS similarity_score
        FROM research_papers
        ORDER BY similarity_score DESC
        FETCH APPROX FIRST {top_k} ROWS ONLY WITH TARGET ACCURACY 90
    """

    # 4️⃣ Execute and return results
    with conn.cursor() as cur:
        cur.execute(query, q=query_embedding_array)
        rows = cur.fetchall()
        columns = [desc[0] for desc in cur.description]

    return rows, columns


In [ ]:
rows, columns = vector_search_research_papers(conn, embedding_model, SEARCH_TEXT_KEYWORDS, top_k=5)

results_df = pd.DataFrame(rows, columns=columns)

print(f"🔍 Vector Search: '{SEARCH_TEXT_KEYWORDS}'")
print(f"📊 Results: {len(results_df)}\n")

results_df


In this step, we used Oracle Database’s native vector search capabilities to perform a semantic similarity search over the research_papers dataset.

The text query — “Get me papers related to planetary exploration” — was first transformed into a high-dimensional embedding vector using the same model that generated our document embeddings (nomic-embed-text-v1.5).
This query vector captures the semantic meaning of the phrase rather than just the exact words.

Using Oracle’s VECTOR_DISTANCE(..., COSINE) function (converted to 1 - distance), we then compared this query vector against the stored embeddings for each paper in the database.
The result is a ranked list of research papers ordered by cosine similarity, where higher scores indicate stronger semantic alignment with the search query.

## 4.3 Hybrid Retrieval (Vector + Text Combined)

#### 4.3.1 Pre Filtering

In [ ]:
import array
import numpy as np

def hybrid_search_research_papers_pre_filter(
    conn,
    embedding_model,
    search_phrase: str,
    top_k: int = 10,
    show_explain: bool = False
):
    """
    Perform a hybrid search using Oracle Text + Vector Search.
    Combines lexical filtering (CONTAINS) with semantic re-ranking via cosine similarity.

    Args:
        conn: Oracle database connection object.
        embedding_model: Model with `.encode()` method (e.g., SentenceTransformer).
        search_phrase (str): User search phrase used for both text filtering and embedding.
        top_k (int): Number of results to return (default = 10).
        show_explain (bool): If True, prints the execution plan.

    Returns:
        tuple: (rows, columns, exec_plan_text or None)
    """

    # --- Step 1: Encode search phrase into normalized vector ---
    query_embedding = embedding_model.encode(
        [f"search_query: {search_phrase}"],
        convert_to_numpy=True,
        normalize_embeddings=True
    )[0].astype(np.float32).tolist()
    query_embedding_array = array.array('f', query_embedding)

    with conn.cursor() as cur:
        # Enable runtime stats if needed
        if show_explain:
            cur.execute("ALTER SESSION SET statistics_level = ALL")

        # --- Step 2: Hybrid query (Oracle Text + Vector) ---
        sql = f"""
            SELECT {"/*+ GATHER_PLAN_STATISTICS */" if show_explain else ""}
                arxiv_id,
                title,
                abstract,
                SUBSTR(text, 1, 200) AS text_snippet,
                ROUND(1 - VECTOR_DISTANCE(embedding, :q, COSINE), 4) AS similarity_score
            FROM research_papers
            WHERE CONTAINS(text, :kw, 1) > 0
            ORDER BY similarity_score DESC
            FETCH APPROX FIRST {top_k} ROWS ONLY WITH TARGET ACCURACY 90
        """

        cur.execute(sql, q=query_embedding_array, kw=search_phrase)
        rows = cur.fetchall()
        columns = [desc[0] for desc in cur.description]

    # --- Step 3: Execution plan (optional) ---
    exec_plan_text = None
    if show_explain:
        with conn.cursor() as cur_plan:
            cur_plan.execute("""
                SELECT plan_table_output
                FROM TABLE(DBMS_XPLAN.DISPLAY_CURSOR(NULL, NULL, 'ALLSTATS LAST +PREDICATE'))
            """)
            exec_plan_text = "\n".join(r[0] for r in cur_plan.fetchall())

        print("\n====== Execution Plan (DBMS_XPLAN.DISPLAY_CURSOR) ======")
        print(exec_plan_text)
        print("========================================================\n")

    return rows, columns, exec_plan_text


In [ ]:
rows, columns, exec_plan = hybrid_search_research_papers_pre_filter(
    conn,
    embedding_model,
    search_phrase=SEARCH_TEXT_KEYWORDS,
    top_k=10,
    show_explain=False
)

pre_filter_results_df = pd.DataFrame(rows, columns=columns)

print(f"🔍 Hybrid Search: '{SEARCH_TEXT_KEYWORDS}'")
print(f"📊 Found {len(pre_filter_results_df)} results\n")

pre_filter_results_df


#### 4.3.2 Post Filtering

In [ ]:
import array
import numpy as np

def hybrid_search_research_papers_postfilter(
    conn,
    embedding_model,
    search_phrase: str,
    top_k: int = 10,
    candidate_k: int = 200,
    show_explain: bool = False
):
    """
    Perform a hybrid search using Vector Search first, then Oracle Text filtering.
    Returns top results ranked by semantic similarity but filtered by lexical match.

    Args:
        conn: Oracle database connection object.
        embedding_model: Model with `.encode()` method (e.g., SentenceTransformer).
        search_phrase (str): Search phrase used for both embedding and text filtering.
        top_k (int): Number of top results to return (default = 10).
        candidate_k (int): Number of initial vector candidates (default = 200).
        show_explain (bool): If True, prints the execution plan.

    Returns:
        tuple: (rows, columns, exec_plan_text or None)
    """

    # --- Step 1: Encode search phrase into a normalized query vector ---
    query_embedding = embedding_model.encode(
        [f"search_query: {search_phrase}"],
        convert_to_numpy=True,
        normalize_embeddings=True
    )[0].astype(np.float32).tolist()
    query_embedding_array = array.array('f', query_embedding)

    with conn.cursor() as cur:
        # Enable runtime statistics if requested
        if show_explain:
            cur.execute("ALTER SESSION SET statistics_level = ALL")

        # --- Step 2: Hybrid query (Vector first → Text filter) ---
        sql = f"""
            WITH vec_candidates AS (
                SELECT
                    arxiv_id,
                    title,
                    abstract,
                    text,
                    1 - VECTOR_DISTANCE(embedding, :q, COSINE) AS similarity_score
                FROM research_papers
                ORDER BY similarity_score DESC
                FETCH APPROX FIRST {candidate_k} ROWS ONLY WITH TARGET ACCURACY 90
            )
            SELECT {"/*+ GATHER_PLAN_STATISTICS */" if show_explain else ""}
                arxiv_id,
                title,
                SUBSTR(text, 1, 200) AS text_snippet,
                ROUND(similarity_score, 4) AS similarity_score
            FROM vec_candidates
            WHERE CONTAINS(text, :kw, 1) > 0
            ORDER BY similarity_score DESC
            FETCH FIRST {top_k} ROWS ONLY
        """

        cur.execute(sql, q=query_embedding_array, kw=search_phrase)
        rows = cur.fetchall()
        columns = [desc[0] for desc in cur.description]

    # --- Step 3: Fetch and display execution plan (optional) ---
    exec_plan_text = None
    if show_explain:
        with conn.cursor() as cur_plan:
            cur_plan.execute("""
                SELECT plan_table_output
                FROM TABLE(DBMS_XPLAN.DISPLAY_CURSOR(NULL, NULL, 'ALLSTATS LAST +PREDICATE'))
            """)
            exec_plan_text = "\n".join(r[0] for r in cur_plan.fetchall())

        print("\n====== Execution Plan (DBMS_XPLAN.DISPLAY_CURSOR) ======")
        print(exec_plan_text)
        print("========================================================\n")

    return rows, columns, exec_plan_text


In [ ]:
rows, columns, exec_plan = hybrid_search_research_papers_postfilter(
    conn,
    embedding_model,
    search_phrase=SEARCH_TEXT_KEYWORDS,
    top_k=10,
    show_explain=False
)

post_filter_results_df = pd.DataFrame(rows, columns=columns)

print(f"🔍 Hybrid Search: '{SEARCH_TEXT_KEYWORDS}'")
print(f"📊 Found {len(post_filter_results_df)} results\n")

post_filter_results_df


Observe pre/post filtering technques in a table side by side 

In [ ]:
# Show the results side by side
pd.concat([pre_filter_results_df, post_filter_results_df], axis=1)


| Approach                          | Strength                            | Best For                                |
| --------------------------------- | ----------------------------------- | --------------------------------------- |
| **Pre-filter** (`CONTAINS` first) | Fast, keyword-strict                | Narrow keyword search                   |
| **Post-filter** (this one)        | Semantically rich but still precise | Broader exploratory or research queries |


#### 4.3.3 Reciprocial Rank Fusion

In [ ]:
import array
import numpy as np
import oracledb

def hybrid_rrf_search(
    conn,
    embedding_model,
    search_phrase: str,
    top_k: int = 10,
    per_list: int = 120,     # candidates from each list before fusion (>= 10x top_k is a good rule)
    k: int = 60,             # RRF smoothing constant (60 is standard)
    phrase_safe: bool = True,
    show_explain: bool = False
):
    """
    Local-friendly RRF fusion of Vector + Oracle Text results on research_papers(text, embedding).

    Prereqs (local/Docker Free OK):
      - VECTOR column/index on research_papers(embedding)  -- HNSW
      - Oracle Text index on research_papers(text)         -- e.g. CREATE SEARCH INDEX rp_text_idx ON research_papers(text);

    RRF = 1/(k + r_vec) + 1/(k + r_txt), where r_vec and r_txt are ranks (1 = best).
    """

    # 1) Encode query for vector modality (align with your doc prefixing scheme)
    qv = embedding_model.encode(
        [f"search_query: {search_phrase}"],
        convert_to_numpy=True,
        normalize_embeddings=True
    )[0].astype(np.float32).tolist()
    qv = array.array('f', qv)

    # 2) Phrase-safe text query for Oracle Text (optional)
    kw = f"\"{search_phrase}\"" if (phrase_safe and " " in search_phrase.strip()) else search_phrase

    with conn.cursor() as cur:
        if show_explain:
            cur.execute("ALTER SESSION SET statistics_level = ALL")

        sql = f"""
            WITH
            /* Vector top-N with ranks (higher similarity first) */
            vec AS (
              SELECT
                arxiv_id,
                title,
                SUBSTR(text, 1, 200) AS text_snippet,
                1 - VECTOR_DISTANCE(embedding, :q, COSINE) AS sim_vec,
                ROW_NUMBER() OVER (ORDER BY 1 - VECTOR_DISTANCE(embedding, :q, COSINE) DESC) AS r_vec
              FROM research_papers
              FETCH APPROX FIRST {per_list} ROWS ONLY WITH TARGET ACCURACY 90
            ),
            /* Oracle Text top-N with ranks (higher SCORE(1) first) */
            txt AS (
              SELECT
                arxiv_id,
                title,
                SUBSTR(text, 1, 200) AS text_snippet,
                SCORE(1) AS score_txt,
                ROW_NUMBER() OVER (ORDER BY SCORE(1) DESC) AS r_txt
              FROM research_papers
              WHERE CONTAINS(text, :kw, 1) > 0
              FETCH FIRST {per_list} ROWS ONLY
            ),
            /* Fuse by arxiv_id; keep docs present in either list */
            fused AS (
              SELECT
                COALESCE(v.arxiv_id, t.arxiv_id)           AS arxiv_id,
                COALESCE(v.title,     t.title)             AS title,
                COALESCE(v.text_snippet, t.text_snippet)   AS text_snippet,
                NVL(v.r_vec,  999999) AS r_vec,
                NVL(t.r_txt,  999999) AS r_txt,
                NVL(v.sim_vec, 0)     AS sim_vec,
                NVL(t.score_txt, 0)   AS score_txt
              FROM vec v
              FULL OUTER JOIN txt t
                ON t.arxiv_id = v.arxiv_id
            )
            SELECT {"/*+ GATHER_PLAN_STATISTICS */" if show_explain else ""}
              arxiv_id,
              title,
              text_snippet,
              ROUND( (1.0/(:k + r_vec)) + (1.0/(:k + r_txt)), 6 ) AS rrf_score,
              r_vec,
              r_txt,
              ROUND(sim_vec, 4)  AS sim_vec,
              ROUND(score_txt,4) AS score_txt
            FROM fused
            ORDER BY rrf_score DESC, title
            FETCH FIRST {top_k} ROWS ONLY
        """

        cur.execute(sql, q=qv, kw=kw, k=k)
        rows = cur.fetchall()
        columns = [d[0] for d in cur.description]

    exec_plan_text = None
    if show_explain:
        with conn.cursor() as cur_plan:
            cur_plan.execute("""
                SELECT plan_table_output
                FROM TABLE(DBMS_XPLAN.DISPLAY_CURSOR(NULL, NULL, 'ALLSTATS LAST +PREDICATE'))
            """)
            exec_plan_text = "\n".join(r[0] for r in cur_plan.fetchall())
            print("\n====== Execution Plan (DBMS_XPLAN.DISPLAY_CURSOR) ======")
            print(exec_plan_text)
            print("========================================================\n")

    return rows, columns, exec_plan_text


In [ ]:
rows, columns, exec_plan = hybrid_rrf_search(
    conn,
    embedding_model,
    search_phrase=SEARCH_TEXT_KEYWORDS,
    top_k=3,
    show_explain=False
)

rrf_results_df = pd.DataFrame(rows, columns=columns)

print(f"🔍 Hybrid Search: '{SEARCH_TEXT_KEYWORDS}'")
print(f"📊 Found {len(rrf_results_df)} results\n")

rrf_results_df


## 4.4 Graph-Based Retrieval (Oracle SQL Property Graph)

In this retrieval mode, we seed with vector similarity, then expand candidates through graph paths:
- `Paper -> SIMILAR_TO -> Paper`
- `Paper <- WROTE - Author - WROTE -> Paper`

The final score blends seed vector relevance and graph-path evidence.


In [ ]:
def graph_search_research_papers(
    conn,
    embedding_model,
    search_query: str,
    top_k: int = 10,
    seed_k: int = 25
):
    """
    Graph retrieval using Oracle SQL Property Graph + GRAPH_TABLE.
    Seeds from vector similarity, then expands via SIMILAR_TO and shared-author paths.
    """

    seed_k = max(seed_k, top_k)

    query_embedding = embedding_model.encode(
        [f"search_query: {search_query}"],
        convert_to_numpy=True,
        normalize_embeddings=True
    )[0].astype(np.float32).tolist()
    
    query_embedding_array = array.array('f', query_embedding)

    sql = f"""
        WITH seed AS (
            SELECT
                arxiv_id,
                1 - VECTOR_DISTANCE(embedding, :q, COSINE) AS seed_score
            FROM research_papers
            ORDER BY seed_score DESC
            FETCH APPROX FIRST {seed_k} ROWS ONLY WITH TARGET ACCURACY 90
        ),
        seed_hits AS (
            SELECT
                arxiv_id AS source_arxiv_id,
                arxiv_id AS candidate_arxiv_id,
                seed_score,
                'seed' AS relation_type,
                seed_score AS edge_score
            FROM seed
        ),
        sim_hops AS (
            SELECT
                s.arxiv_id AS source_arxiv_id,
                gt.target_arxiv_id AS candidate_arxiv_id,
                s.seed_score,
                'similar_to' AS relation_type,
                gt.edge_score AS edge_score
            FROM seed s
            JOIN GRAPH_TABLE(
                research_graph
                MATCH (src IS paper)-[e IS similar_to]->(dst IS paper)
                COLUMNS (
                    src.arxiv_id AS source_arxiv_id,
                    dst.arxiv_id AS target_arxiv_id,
                    e.sim_score AS edge_score
                )
            ) gt
                ON gt.source_arxiv_id = s.arxiv_id
        ),
        author_hops AS (
            SELECT
                s.arxiv_id AS source_arxiv_id,
                gt.target_arxiv_id AS candidate_arxiv_id,
                s.seed_score,
                'shared_author' AS relation_type,
                1.0 AS edge_score
            FROM seed s
            JOIN GRAPH_TABLE(
                research_graph
                MATCH (src IS paper)<-[w1 IS wrote]-(a IS author)-[w2 IS wrote]->(dst IS paper)
                COLUMNS (
                    src.arxiv_id AS source_arxiv_id,
                    dst.arxiv_id AS target_arxiv_id
                )
            ) gt
                ON gt.source_arxiv_id = s.arxiv_id
            WHERE gt.target_arxiv_id <> s.arxiv_id
        ),
        candidates AS (
            SELECT * FROM seed_hits
            UNION ALL
            SELECT * FROM sim_hops
            UNION ALL
            SELECT * FROM author_hops
        ),
        scored AS (
            SELECT
                candidate_arxiv_id AS arxiv_id,
                MAX(
                    CASE relation_type
                        WHEN 'seed' THEN seed_score
                        WHEN 'similar_to' THEN (0.70 * seed_score) + (0.30 * edge_score)
                        WHEN 'shared_author' THEN (0.85 * seed_score) + (0.15 * edge_score)
                        ELSE seed_score
                    END
                ) AS graph_score
            FROM candidates
            GROUP BY candidate_arxiv_id
        )
        SELECT
            rp.arxiv_id,
            rp.title,
            rp.abstract,
            SUBSTR(rp.text, 1, 200) AS text_snippet,
            ROUND(sc.graph_score, 4) AS graph_score
        FROM scored sc
        JOIN research_papers rp
            ON rp.arxiv_id = sc.arxiv_id
        ORDER BY graph_score DESC
        FETCH FIRST {top_k} ROWS ONLY
    """

    with conn.cursor() as cur:
        cur.execute(sql, q=query_embedding_array)
        rows = cur.fetchall()
        columns = [desc[0] for desc in cur.description]

    return rows, columns


This quick check runs graph retrieval on the same sample query so you can compare graph-ranked results side-by-side with keyword/vector/hybrid modes.


In [ ]:
rows, columns = graph_search_research_papers(
    conn,
    embedding_model,
    search_query=SEARCH_TEXT_KEYWORDS,
    top_k=5,
    seed_k=20
)

graph_results_df = pd.DataFrame(rows, columns=columns)

print(f"🔍 Graph Search: '{SEARCH_TEXT_KEYWORDS}'")
print(f"📊 Results: {len(graph_results_df)}\n")

graph_results_df


### 4.5 Compare Top Results Across Retrieval Strategies

To make the retrieval behavior easy to inspect, the next cell builds a side-by-side table of titles.

Each row is a retrieval strategy, and the columns `Top_1` to `Top_5` show the highest-ranked paper titles returned for the same query.


In [ ]:
def extract_top_titles(rows, columns, k=5):
    """Return top-k titles from retrieval results."""
    titles = []
    for row in rows[:k]:
        row_data = dict(zip(columns, row))
        titles.append(row_data.get("TITLE", "Untitled"))
    return titles


comparison_specs = [
    ("keyword", lambda: keyword_search_research_papers(conn, SEARCH_TEXT_KEYWORDS)),
    ("vector", lambda: vector_search_research_papers(conn, embedding_model, SEARCH_TEXT_KEYWORDS, top_k=5)),
    (
        "hybrid_pre_filter",
        lambda: hybrid_search_research_papers_pre_filter(
            conn=conn,
            embedding_model=embedding_model,
            search_phrase=SEARCH_TEXT_KEYWORDS,
            top_k=5,
            show_explain=False,
        ),
    ),
    (
        "hybrid_postfilter",
        lambda: hybrid_search_research_papers_postfilter(
            conn=conn,
            embedding_model=embedding_model,
            search_phrase=SEARCH_TEXT_KEYWORDS,
            top_k=5,
            candidate_k=200,
            show_explain=False,
        ),
    ),
    (
        "hybrid_rrf",
        lambda: hybrid_rrf_search(
            conn=conn,
            embedding_model=embedding_model,
            search_phrase=SEARCH_TEXT_KEYWORDS,
            top_k=5,
            per_list=60,
            k=60,
            show_explain=False,
        ),
    ),
    (
        "graph",
        lambda: graph_search_research_papers(
            conn=conn,
            embedding_model=embedding_model,
            search_query=SEARCH_TEXT_KEYWORDS,
            top_k=5,
            seed_k=20,
        ),
    ),
]

comparison_rows = []
for strategy_name, runner in comparison_specs:
    result = runner()

    if isinstance(result, tuple) and len(result) >= 2:
        rows, columns = result[0], result[1]
    else:
        rows, columns = [], []

    titles = extract_top_titles(rows, columns, k=5)

    record = {"retrieval_strategy": strategy_name}
    for i in range(5):
        record[f"Top_{i+1}"] = titles[i] if i < len(titles) else ""

    comparison_rows.append(record)

retrieval_strategy_comparison_df = pd.DataFrame(comparison_rows)
retrieval_strategy_comparison_df


# Part 5: Building a RAG Pipeline

In this section, we connect retrieval to generation end-to-end.

Flow for this part:
1. Configure API access and initialize the OpenAI client.
2. Define a reusable RAG function that supports multiple retrieval modes.
3. Retrieve evidence from Oracle and synthesize a grounded answer.
4. Run a sample query to validate the pipeline behavior.


### 5.1 Configure API Access

The next two cells define a small helper to securely set environment variables and prompt for `OPENAI_API_KEY`. This keeps credentials out of source code and notebook outputs.


In [ ]:
import getpass
import os

# Function to securely get and set environment variables
def set_env_securely(var_name, prompt):
    value = getpass.getpass(prompt)
    os.environ[var_name] = value

In [ ]:
set_env_securely("OPENAI_API_KEY", "Enter your OPEN API Key: ")

### 5.2 Initialize and Smoke-Test the OpenAI Client

Before building the full pipeline, we perform a lightweight client call to confirm model access and credential configuration.


In [ ]:
# Import the OpenAI Python client library
from openai import OpenAI
import os

# Initialize the OpenAI client (API key read from env var OPENAI_API_KEY)
openai_client = OpenAI(
    api_key=os.environ.get("OPENAI_API_KEY"),
)

# Use the Responses API
response = openai_client.responses.create(
    model="gpt-5",
    input="Hello! I’m a user!",
    instructions="You are a research paper assistant.",
)

In [ ]:
# Print the output text
print(response.output_text)

### 5.3 Define the Reusable RAG Function

This function is the core integration point between retrieval and generation. It:
- selects retrieval strategy (`keyword`, `vector`, `hybrid`, or `graph`),
- formats retrieved rows into citation-ready context,
- calls the Responses API with grounding instructions.


In [ ]:
def research_paper_assistant_rag_pipeline(
    conn,
    embedding_model,
    user_query: str,
    top_k: int = 10,
    retrieval_mode: str = "hybrid",
    show_explain: bool = False
):
    """
    Research Paper Assistant — Retrieval-Augmented Generation (RAG) pipeline
    built on SQL-based retrieval functions and powered by the OpenAI Responses API.

    Retrieval techniques available:
        - 'keyword'  → uses keyword_search_research_papers()
        - 'vector'   → uses vector_search_research_papers()
        - 'hybrid'   → uses hybrid_search_research_papers_pre_filter() [default]
        - 'graph'    → uses graph_search_research_papers() via SQL Property Graph

    Args:
        conn: Oracle database connection.
        embedding_model: Embedding model (e.g., SentenceTransformer, Voyage).
        user_query (str): Research question from the user.
        top_k (int): Number of top documents to retrieve.
        retrieval_mode (str): Retrieval method ('keyword', 'vector', 'hybrid', 'graph').
        show_explain (bool): Whether to show the SQL execution plan.

    Returns:
        str: LLM-generated research synthesis with citations.
    """

    # ----------------------------------------------------------------------
    # 1. Retrieve relevant research papers using the selected retrieval mode
    # ----------------------------------------------------------------------
    if retrieval_mode == "keyword":
        rows, columns = keyword_search_research_papers(conn, user_query)
        exec_plan_text = None

    elif retrieval_mode == "vector":
        rows, columns = vector_search_research_papers(conn, embedding_model, user_query, top_k)
        exec_plan_text = None

    elif retrieval_mode == "graph":
        rows, columns = graph_search_research_papers(conn, embedding_model, user_query, top_k=top_k)
        exec_plan_text = None

    else:  # default: hybrid retrieval
        rows, columns, exec_plan_text = hybrid_search_research_papers_pre_filter(
            conn=conn,
            embedding_model=embedding_model,
            search_phrase=user_query,
            top_k=top_k,
            show_explain=show_explain
        )

    retrieved_count = len(rows) if rows else 0
    print(f"📊 Retrieved {retrieved_count} papers using {retrieval_mode.upper()} retrieval.")


    # ----------------------------------------------------------------------
    # 2. Convert retrieved rows to formatted LLM context
    # ----------------------------------------------------------------------
    formatted_context = ""
    if retrieved_count > 0:
        formatted_context += f"\n\n📚 {retrieved_count} relevant research papers retrieved:\n\n"
        for i, row in enumerate(rows):
            row_data = dict(zip(columns, row))
            title = row_data.get("TITLE", "Untitled Paper")
            abstract = row_data.get("ABSTRACT", "No abstract available.")
            snippet = row_data.get("TEXT_SNIPPET", "")
            score = (
                row_data.get("GRAPH_SCORE")
                or row_data.get("SIMILARITY_SCORE")
                or row_data.get("RELEVANCE_SCORE")
                or row_data.get("TEXT_RELEVANCE_SCORE")
                or "N/A"
            )
            formatted_context += (
                f"[{i+1}] **{title}**\n"
                f"Abstract: {abstract}\n"
                f"Snippet: {snippet}\n"
                f"Relevance Score: {score}\n\n"
            )
    else:
        formatted_context = "\n\n⚠️ No relevant papers were retrieved from the database.\n"

    # ----------------------------------------------------------------------
    # 3. Construct the prompt for the Responses API
    # ----------------------------------------------------------------------
    prompt = f"""
            You are a **Research Paper Assistant** that synthesizes academic literature to help answer user questions.

            User Query: {user_query}

            Number of retrieved papers: {retrieved_count}
            {formatted_context}

            Please:
            - Summarize the findings most relevant to the query.
            - Use citation numbers [X] to support claims.
            - Highlight consensus, innovation, or research gaps.
            - If there is insufficient context, clearly say so.
            """

    # ----------------------------------------------------------------------
    # 4. Call the OpenAI Responses API
    # ----------------------------------------------------------------------
    response = openai_client.responses.create(
        model="gpt-4o",
        input=prompt,
        instructions="You are a scientific research assistant. Use only the provided context to answer. Always cite papers [1], [2], etc.",
        temperature=0.3,
    )

    # ----------------------------------------------------------------------
    # 5. Optionally print SQL execution plan (if hybrid)
    # ----------------------------------------------------------------------
    if show_explain and exec_plan_text:
        print("\n====== SQL Execution Plan ======")
        print(exec_plan_text)
        print("================================\n")

    # ----------------------------------------------------------------------
    # 6. Return the LLM’s output text
    # ----------------------------------------------------------------------
    return response.output_text


### 5.4 Run an End-to-End RAG Example

This invocation exercises the full path from user query to retrieval to synthesized answer, so you can validate retrieval quality and response grounding.


In [ ]:
summary = research_paper_assistant_rag_pipeline(
    conn=conn,
    embedding_model=embedding_model,
    user_query=SEARCH_TEXT_KEYWORDS,
    top_k=5,
    retrieval_mode="hybrid",  # options: 'keyword', 'vector', 'hybrid', 'graph'
    show_explain=False
)

print(summary)


# Part 6: AI Agents with OpenAI and Oracle AI Database

In this section, we move from a single RAG call to an agentic system.

You will build the flow incrementally:
1. Start with a base agent.
2. Add retrieval tools backed by Oracle SQL.
3. Upgrade to multi-tool routing and orchestration.
4. Add conversation history and persistent session memory.


### 6.1 Install Agent Runtime and Verify Package Compatibility

Use the notebook-native `%pip` command so installation happens in the **active kernel environment**.

We install `openai` and `openai-agents` together to avoid version mismatches, then print resolved versions before importing `agents`.

If you still see import errors after install, restart the kernel and run this section again.


In [ ]:
%pip install -Uq --no-cache-dir openai openai-agents

In [ ]:
import sys
import importlib.metadata as md

print("Python executable:", sys.executable)
print("openai version:", md.version("openai"))
print("openai-agents version:", md.version("openai-agents"))

OPENAI_MODEL = "gpt-5"


### 6.2 Create a Baseline Agent (No Tools Yet)

Start simple: define a research-focused assistant and run one direct query. This establishes baseline behavior before enabling external tool use.


In [ ]:
from agents import Agent, Runner

research_paper_assistant = Agent(
    name="Research Paper Assistant",
    model=OPENAI_MODEL,
    instructions="""
      You are a Research Paper Assistant focused on helping users explore, analyze, and summarize
      academic research.

      Maintain a professional, concise, and scholarly tone appropriate for research discussions.
    """,
)


In [ ]:
run_result = await Runner.run(
    starting_agent=research_paper_assistant,
    input="Summarize recent research on optimization techniques for planetary exploration.",
)

In [ ]:
print(run_result.final_output)


### 6.3 Expose Oracle Retrieval as a Callable Tool

Here we wrap SQL retrieval in a `@function_tool`, attach it to the assistant, and run a query that should trigger tool usage. Inspecting raw responses helps verify tool-call arguments and execution order.


In [ ]:
from agents.tool import function_tool

@function_tool
def get_research_papers(user_query: str, retrieval_mode: str = "hybrid", top_k: int = 5) -> str:
    """
    Retrieves academic research papers relevant to the user's query.

    This tool queries the research_papers SQL table using one of four retrieval techniques:
        - 'keyword'  → lexical search via Oracle Text
        - 'vector'   → semantic similarity search
        - 'hybrid'   → combines keyword prefiltering + vector similarity (default)
        - 'graph'    → graph expansion via SQL Property Graph + GRAPH_TABLE

    Use this tool when analyzing or summarizing scientific literature.

    Args:
        user_query (str): Research topic or question to search for.
        retrieval_mode (str): 'keyword', 'vector', 'hybrid', or 'graph'. Default is 'hybrid'.
        top_k (int): Number of top papers to retrieve (default=5).

    Returns:
        str: A formatted summary of the most relevant research papers.
    """

    # ------------------------------------------------------------------
    # Perform retrieval using SQL-based functions (defined earlier)
    # ------------------------------------------------------------------
    if retrieval_mode == "keyword":
        rows, columns = keyword_search_research_papers(conn, user_query)
    elif retrieval_mode == "vector":
        rows, columns = vector_search_research_papers(conn, embedding_model, user_query, top_k)
    elif retrieval_mode == "graph":
        rows, columns = graph_search_research_papers(conn, embedding_model, user_query, top_k=top_k)
    else:
        rows, columns, _ = hybrid_search_research_papers_pre_filter(
            conn=conn,
            embedding_model=embedding_model,
            search_phrase=user_query,
            top_k=top_k,
            show_explain=False
        )

    retrieved_count = len(rows) if rows else 0

    # ------------------------------------------------------------------
    # Format the output into a readable string
    # ------------------------------------------------------------------
    if retrieved_count == 0:
        return f"No research papers found related to '{user_query}'."

    formatted_results = [f"📚 {retrieved_count} papers retrieved for query: '{user_query}'\n"]
    for i, row in enumerate(rows):
        row_data = dict(zip(columns, row))
        title = row_data.get("TITLE", "Untitled Paper")
        abstract = row_data.get("ABSTRACT", "No abstract available.")
        score = (
            row_data.get("GRAPH_SCORE")
            or row_data.get("SIMILARITY_SCORE")
            or row_data.get("RELEVANCE_SCORE")
            or row_data.get("TEXT_RELEVANCE_SCORE")
            or "N/A"
        )
        formatted_results.append(
            f"[{i+1}] {title}\n"
            f"Abstract: {abstract}\n"
            f"Relevance Score: {score}\n"
        )

    return "\n".join(formatted_results)


In [ ]:
research_paper_assistant.tools.append(get_research_papers)

In [ ]:
run_result_with_tool = await Runner.run(
    starting_agent=research_paper_assistant,
    input="Get me information on rover navigation, planetary data collection, mission planning, resource allocation, or other related fields",
)

In [ ]:
print(run_result_with_tool.final_output)

In [ ]:
import pprint
pprint.pprint(run_result_with_tool.raw_responses)

### 6.4 Add a Second Tool and Enable Multi-Tool Access

The first tool retrieves current literature. In this block, we add a second tool for past research conversations so the assistant can combine:
- fresh evidence from the paper corpus, and
- prior analytical context from earlier runs.

This is the point where the agent starts making tool-selection decisions based on intent.


In [ ]:
from agents.tool import function_tool

@function_tool
def get_past_research_conversations(user_query: str, top_k: int = 5) -> str:
    """
    Retrieves relevant past research-related conversations or analyses related to the query.

    This tool searches a SQL database of prior research assistant conversations, 
    literature discussions, or synthesis sessions to find relevant context. 
    It allows the research assistant to recall previous analyses or summaries 
    that addressed similar topics, providing continuity and richer insights.

    Args:
        user_query (str): The research topic, concept, or question to search for.
        top_k (int): Number of top past discussions to retrieve (default=5).

    Returns:
        str: Formatted examples of relevant past research discussions.
    """

    # ------------------------------------------------------------------
    # Perform retrieval using the SQL-based hybrid search (vector + keyword)
    # ------------------------------------------------------------------
    rows, columns, _ = hybrid_search_research_papers_pre_filter(
        conn=conn,
        embedding_model=embedding_model,
        search_phrase=user_query,
        top_k=top_k,
        show_explain=False
    )

    retrieved_count = len(rows) if rows else 0

    # ------------------------------------------------------------------
    # Format results for readability
    # ------------------------------------------------------------------
    if retrieved_count == 0:
        return f"No past research discussions found related to '{user_query}'."

    formatted_results = [f"🧠 {retrieved_count} past research discussions retrieved for query: '{user_query}'\n"]
    for i, row in enumerate(rows):
        row_data = dict(zip(columns, row))
        title = row_data.get("TITLE", "Untitled Discussion")
        abstract = row_data.get("ABSTRACT", "No summary available.")
        snippet = row_data.get("TEXT_SNIPPET", "")
        score = (
            row_data.get("SIMILARITY_SCORE")
            or row_data.get("TEXT_RELEVANCE_SCORE")
            or "N/A"
        )
        formatted_results.append(
            f"[{i+1}] **{title}**\n"
            f"Summary: {abstract}\n"
            f"Snippet: {snippet}\n"
            f"Relevance Score: {score}\n"
        )

    return "\n".join(formatted_results)


### 6.5 Strengthen Agent Instructions for Tool Routing

Tool availability alone is not enough. The instruction policy below makes routing explicit so the assistant knows:
- when to call paper retrieval,
- when to call conversation retrieval,
- and when to combine both for broader synthesis.


In [ ]:
upgraded_research_paper_assistant = Agent(
    name="Research Paper Assistant",
    model=OPENAI_MODEL,
    instructions="""
    Always maintain an academic, evidence-based tone.
    Your purpose is to help users explore, synthesize, and connect research insights —
    not to speculate or fabricate information.
    """,
)


In [ ]:
# Attach research retrieval tools to the upgraded research assistant
upgraded_research_paper_assistant.tools.append(get_research_papers)
upgraded_research_paper_assistant.tools.append(get_past_research_conversations)

In [ ]:
pprint.pprint(upgraded_research_paper_assistant.tools)

In [ ]:
run_result_with_tools = await Runner.run(
    starting_agent=upgraded_research_paper_assistant,
    input=(
        "Get me information on rover navigation, planetary data collection, mission planning, resource allocation, or other related fields "
    ),
)

In [ ]:
print(run_result_with_tools.raw_responses)

## Agent-as-Tools Orchestration

Now we compose specialists:
- a research-paper retrieval specialist,
- a past-conversation retrieval specialist,
- an orchestrator that delegates,
- and a final synthesizer that produces a cohesive response.

This mirrors a production pattern where narrow agents perform focused work and a coordinator merges outputs.


### Orchestration Flow

Execution sequence in the next cells:
1. Define specialized agents with clear scope.
2. Register them as tools on an orchestrator agent.
3. Run the orchestrator to gather relevant evidence.
4. Pass gathered evidence to a synthesizer for final answer generation.

The benefit is separation of concerns: retrieval logic is modular, and synthesis is centralized.


In [ ]:
# Define specialized agents for different research retrieval tasks
research_paper_agent = Agent(
    name="research_paper_agent",
    instructions="""
        You specialize in retrieving and summarizing academic research papers.
        Use the get_research_papers tool to find relevant literature based on the user's query.
        Always cite sources using [1], [2], etc., and focus on summarizing key findings,
        methodologies, and implications of the studies retrieved.
    """,
    handoff_description="A research retrieval specialist with access to academic papers and literature databases.",
    tools=[get_research_papers],
)

research_conversation_agent = Agent(
    name="research_conversation_agent",
    instructions="""
        You specialize in retrieving and summarizing past research discussions and analyses.
        Use the get_past_research_conversations tool to surface relevant prior sessions
        or summaries that relate to the user's current topic of inquiry.
        Present these as context and examples of prior analytical reasoning.
    """,
    handoff_description="A research memory specialist with access to prior academic discussions and analyses.",
    tools=[get_past_research_conversations],
)


In [ ]:
# Create an orchestrator agent that can coordinate both research retrieval agents
orchestrator_agent = Agent(
    name="research_assistant_orchestrator",
    instructions=(
        "You are a Research Orchestrator Assistant responsible for coordinating information retrieval "
        "across multiple specialized research tools.\n\n"
        "Your role is to help users explore, analyze, and synthesize academic research efficiently.\n\n"
        "IMPORTANT RULES:\n"
        "1. ALWAYS use translate_to_research_papers when a query mentions research papers, studies, or findings.\n"
        "2. ALWAYS use translate_to_research_conversations when a query mentions previous discussions, analyses, or summaries.\n"
        "3. If a query requests BOTH new research and past discussions, use BOTH tools in sequence.\n"
        "4. NEVER attempt to provide research summaries without using your tools.\n"
        "5. Each tool provides complementary context — use all appropriate tools for a comprehensive academic response.\n\n"
        "After retrieving relevant results, synthesize them into a cohesive summary:\n"
        "- Clearly distinguish between newly retrieved research and recalled past discussions.\n"
        "- Cite sources using [1], [2], etc.\n"
        "- Identify key insights, trends, and research gaps.\n"
        "- Maintain an academic and objective tone."
    ),
    tools=[
        research_paper_agent.as_tool(
            tool_name="translate_to_research_papers",
            tool_description="Retrieve and summarize relevant academic research papers and literature findings.",
        ),
        research_conversation_agent.as_tool(
            tool_name="translate_to_research_conversations",
            tool_description="Retrieve and summarize past research discussions or analyses related to the topic.",
        ),
    ],
)


In [ ]:
# Final agent to synthesize information from all sources (Research use case)
synthesizer_agent = Agent(
    name="research_response_synthesizer",
    instructions=(
        "You create comprehensive, well-organized research summaries by combining information from multiple sources.\n\n"
        "When organizing your response:\n"
        "1) Start with a concise abstract-style overview (3–5 sentences) highlighting key findings and takeaways.\n"
        "2) Clearly separate NEW LITERATURE FINDINGS from PAST RESEARCH DISCUSSIONS.\n"
        "3) Cite sources using bracketed numbers [1], [2], etc., aligned with the retrieved items.\n"
        "4) Emphasize methods, evidence strength, and limitations; avoid speculation beyond the provided context.\n"
        "5) Use clear, scannable formatting (short paragraphs, bullet points where appropriate).\n"
        "6) Conclude with open questions, gaps, or future work suggested by the literature.\n"
        "7) If evidence is sparse, state this explicitly and avoid overgeneralization.\n"
        "Tone: academic, objective, and precise."
    ),
)


### 6.6 Run the Asynchronous Orchestration Workflow

The next cells define async workflow execution, patch the notebook event loop, and provide a synchronous wrapper for interactive use.


In [ ]:
from agents import ItemHelpers, MessageOutputItem, trace
from agents import Runner  # assuming Runner is imported elsewhere; include here for clarity


async def research_assistant_workflow(user_query: str):
    """Run the complete research assistant workflow (orchestrate retrieval + synthesize)."""
    # 1) Have the research orchestrator decide which tools to invoke
    with trace("Research Orchestrator"):
        orchestrator_result = await Runner.run(orchestrator_agent, user_query)

        # Debug/transparency: print intermediate orchestration steps
        print("\n--- Research Orchestration Steps ---")
        for item in orchestrator_result.new_items:
            if isinstance(item, MessageOutputItem):
                text = ItemHelpers.text_message_output(item)
                if text:
                    print(f"  - Retrieval step: {text}")

        # 2) Synthesize all gathered information into a cohesive research summary
        synthesizer_result = await Runner.run(
            synthesizer_agent, orchestrator_result.to_input_list()
        )

        print(f"\n\n--- Final Research Synthesis ---\n{synthesizer_result.final_output}\n")

    return synthesizer_result.final_output


In [ ]:
import asyncio
import nest_asyncio

# Apply nest_asyncio to patch the event loop
nest_asyncio.apply()

In [ ]:
def run_virtual_research_assistant(query):
    # Create a new event loop
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)

    # Run the async function and get the result
    result = loop.run_until_complete(research_assistant_workflow(query))

    # Clean up
    loop.close()

    return result

In [ ]:
# Now call the function this way
query = input("What research topic can I help you with today? ")
run_virtual_research_assistant(query)

## Agentic Chat System

The next block adds thread-aware conversation handling on top of orchestration.

For each user turn, the system:
1. Stores the new message in Oracle.
2. Reconstructs conversation context by `thread_id`.
3. Runs orchestration + synthesis with that context.
4. Saves the assistant response back to Oracle.

This pattern supports long-running research sessions with continuity.


In [ ]:
import datetime
import uuid

# Create chat_history table in Oracle
with conn.cursor() as cur:
    # Drop table if exists (for development)
    cur.execute("""
        BEGIN
            EXECUTE IMMEDIATE 'DROP TABLE chat_history';
        EXCEPTION WHEN OTHERS THEN
            IF SQLCODE != -942 THEN RAISE; END IF;
        END;
    """)
    
    # Create chat_history table
    cur.execute("""
        CREATE TABLE chat_history (
            id VARCHAR2(100) PRIMARY KEY,
            thread_id VARCHAR2(100) NOT NULL,
            role VARCHAR2(20) NOT NULL,
            message CLOB NOT NULL,
            timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
        )
        TABLESPACE USERS
    """)
    
    # Create index on thread_id and timestamp for efficient retrieval
    cur.execute("""
        CREATE INDEX idx_thread_timestamp 
        ON chat_history(thread_id, timestamp)
        TABLESPACE USERS
    """)
    
    conn.commit()
    print("✅ Table chat_history created successfully with index.")

In [ ]:
async def research_assistant_chat(user_query, thread_id=None):
    """
    Run the complete research assistant workflow with conversation history.
    For each conversation turn:
      - Stores the user's input and the assistant's output in Oracle along with a timestamp and thread_id.
      - Retrieves and appends previous conversation history (ordered by timestamp) to the agent's input.
    
    If no thread_id is provided, a new conversation session is started.
    
    Returns:
      tuple: (final_output, thread_id) where thread_id is the session identifier.
    """
    # Generate a new thread id if not provided
    if thread_id is None:
        thread_id = str(uuid.uuid4())
        print(f"📝 New research conversation started with thread ID: {thread_id}")
    else:
        print(f"📝 Continuing research conversation with thread ID: {thread_id}")
    
    # --- Step 1: Store the new user query in Oracle ---
    message_id = str(uuid.uuid4())
    
    with conn.cursor() as cur:
        cur.execute("""
            INSERT INTO chat_history (id, thread_id, role, message, timestamp)
            VALUES (:id, :thread_id, :role, :message, CURRENT_TIMESTAMP)
        """, {
            'id': message_id,
            'thread_id': thread_id,
            'role': 'user',
            'message': user_query
        })
        conn.commit()
    
    # --- Step 2: Retrieve full conversation history for context ---
    with conn.cursor() as cur:
        cur.execute("""
            SELECT role, message, timestamp
            FROM chat_history
            WHERE thread_id = :thread_id
            ORDER BY timestamp ASC
        """, {'thread_id': thread_id})
        
        chat_history = cur.fetchall()
    
    conversation_context = ""
    for entry in chat_history:
        role, message, timestamp = entry
        if role == "user":
            conversation_context += f"User: {message}\n"
        else:
            conversation_context += f"Assistant: {message}\n"
    
    # --- Step 3: Run the orchestrator agent with the conversation context ---
    with trace("Research Orchestrator"):
        orchestrator_result = await Runner.run(orchestrator_agent, conversation_context)
    
    # Print intermediate processing steps for debugging/transparency
    print("\n--- Research Orchestrator Processing Steps ---")
    for item in orchestrator_result.new_items:
        if isinstance(item, MessageOutputItem):
            text = ItemHelpers.text_message_output(item)
            if text:
                print(f"  - Information gathering step: {text}")
    
    # --- Step 4: Run the synthesizer agent to produce a cohesive response ---
    synthesizer_result = await Runner.run(
        synthesizer_agent, orchestrator_result.to_input_list()
    )
    
    # --- Step 5: Store the assistant's final output in Oracle ---
    response_id = str(uuid.uuid4())
    
    with conn.cursor() as cur:
        cur.execute("""
            INSERT INTO chat_history (id, thread_id, role, message, timestamp)
            VALUES (:id, :thread_id, :role, :message, CURRENT_TIMESTAMP)
        """, {
            'id': response_id,
            'thread_id': thread_id,
            'role': 'assistant',
            'message': synthesizer_result.final_output
        })
        conn.commit()
    
    print(f"\n\n--- Final Research Response ---\n{synthesizer_result.final_output}\n")
    
    return synthesizer_result.final_output, thread_id

In [ ]:
def run_research_assistant_chat(query, thread_id=None):
    """
    Run the research assistant synchronously.
    Optionally, a thread_id can be provided to continue an existing conversation.
    Returns a tuple (final_output, thread_id).
    """
    # Create a new event loop
    loop = asyncio.new_event_loop()
    asyncio.set_event_loop(loop)
    
    # Run the async function and get the result
    result, thread_id = loop.run_until_complete(
        research_assistant_chat(query, thread_id=thread_id)
    )
    
    # Clean up the loop
    loop.close()
    
    return result, thread_id

In [ ]:
def research_chat_session():
    """
    Launches a research chat session that continues until the user enters 'q', 'exit', or 'quit'.
    The session uses a persistent thread_id to preserve conversation history.
    """
    print("🔬 Starting Research Paper Assistant Chat")
    print("Type 'q', 'exit' or 'quit' to exit.\n")
    
    session_thread_id = None
    
    while True:
        query = input("What research topic can I help you with today? ")
        
        if query.lower() in ["q", "exit", "quit"]:
            print("Exiting research chat session.")
            break
        
        response, session_thread_id = run_research_assistant_chat(
            query, thread_id=session_thread_id
        )
        
        print(f"\n📚 Assistant: {response}\n")

In [ ]:
# Start the research chat session
research_chat_session()

## Session Memory with Oracle AI Database

This section implements a custom `OracleSession` adapter compatible with agent session APIs.

Key capabilities:
- persistent storage of message items,
- retrieval in chronological order,
- controlled trimming (`pop_item`) to manage memory budget,
- full session reset (`clear_session`) when needed.


In [ ]:
from typing import List, Optional, Union
from datetime import datetime
import oracledb
import json
import uuid

class OracleSession:
    """Custom Oracle session implementation following the Session protocol"""
    
    def __init__(
        self, 
        session_id: str, 
        connection,
        table_name: str = "chat_history"
    ):
        """
        Initialize Oracle session storage.
        
        Args:
            session_id: Unique identifier for this conversation session
            connection: Active oracledb connection object
            table_name: Name of the Oracle table storing session data
        """
        self.session_id = session_id
        self.conn = connection
        self.table_name = table_name
    
    async def get_items(self, limit: Optional[int] = None) -> List[dict]:
        """Retrieve conversation history for this session"""
        try:
            with self.conn.cursor() as cur:
                if limit:
                    cur.execute(f"""
                        SELECT message
                        FROM {self.table_name}
                        WHERE thread_id = :session_id
                        ORDER BY timestamp ASC
                        FETCH FIRST :limit ROWS ONLY
                    """, {'session_id': self.session_id, 'limit': limit})
                else:
                    cur.execute(f"""
                        SELECT message
                        FROM {self.table_name}
                        WHERE thread_id = :session_id
                        ORDER BY timestamp ASC
                    """, {'session_id': self.session_id})
                
                rows = cur.fetchall()
                
                items = []
                for row in rows:
                    # Deserialize JSON from CLOB
                    message_clob = row[0]
                    if message_clob:
                        message_str = message_clob.read() if hasattr(message_clob, 'read') else str(message_clob)
                        items.append(json.loads(message_str))
                
                return items
        
        except Exception as e:
            print(f"Error retrieving items: {e}")
            return []
    
    async def add_items(self, items: List[dict]) -> None:
        """Store new items for this session"""
        try:
            with self.conn.cursor() as cur:
                for item in items:
                    item_id = str(uuid.uuid4())
                    
                    # Serialize the entire item as JSON
                    message_json = json.dumps(item)
                    
                    # Extract role if available, otherwise default to 'system'
                    role = item.get('role', 'system')
                    
                    cur.execute(f"""
                        INSERT INTO {self.table_name} (id, thread_id, role, message, timestamp)
                        VALUES (:id, :session_id, :role, :message, CURRENT_TIMESTAMP)
                    """, {
                        'id': item_id,
                        'session_id': self.session_id,
                        'role': role,
                        'message': message_json
                    })
                
                self.conn.commit()
        
        except Exception as e:
            print(f"Error adding items: {e}")
            self.conn.rollback()
    
    async def pop_item(self, limit: Optional[int] = None) -> Optional[Union[dict, List[dict]]]:
        """
        Remove and return the most recent item(s) for this session.
        """
        try:
            with self.conn.cursor() as cur:
                # Pop a single most-recent item
                if not limit or limit <= 1:
                    cur.execute(f"""
                        SELECT id, message
                        FROM {self.table_name}
                        WHERE thread_id = :session_id
                        ORDER BY timestamp DESC
                        FETCH FIRST 1 ROW ONLY
                    """, {'session_id': self.session_id})
                    
                    row = cur.fetchone()
                    
                    if row:
                        item_id, message_clob = row
                        message_str = message_clob.read() if hasattr(message_clob, 'read') else str(message_clob)
                        item = json.loads(message_str)
                        
                        # Delete the item
                        cur.execute(f"""
                            DELETE FROM {self.table_name}
                            WHERE id = :id
                        """, {'id': item_id})
                        
                        self.conn.commit()
                        return item
                    
                    return None
                
                # Pop multiple most-recent items
                cur.execute(f"""
                    SELECT id, message
                    FROM {self.table_name}
                    WHERE thread_id = :session_id
                    ORDER BY timestamp DESC
                    FETCH FIRST :limit ROWS ONLY
                """, {'session_id': self.session_id, 'limit': limit})
                
                rows = cur.fetchall()
                
                if not rows:
                    return []
                
                items = []
                ids_to_delete = []
                
                for row in rows:
                    item_id, message_clob = row
                    message_str = message_clob.read() if hasattr(message_clob, 'read') else str(message_clob)
                    items.append(json.loads(message_str))
                    ids_to_delete.append(item_id)
                
                # Delete all items
                for item_id in ids_to_delete:
                    cur.execute(f"""
                        DELETE FROM {self.table_name}
                        WHERE id = :id
                    """, {'id': item_id})
                
                self.conn.commit()
                return items
        
        except Exception as e:
            print(f"Error popping item(s): {e}")
            self.conn.rollback()
            return None if (not limit or limit <= 1) else []
    
    async def clear_session(self) -> None:
        """Clear all items for this session"""
        try:
            with self.conn.cursor() as cur:
                cur.execute(f"""
                    DELETE FROM {self.table_name}
                    WHERE thread_id = :session_id
                """, {'session_id': self.session_id})
                
                self.conn.commit()
                print(f"✅ Session {self.session_id} cleared successfully.")
        
        except Exception as e:
            print(f"Error clearing session: {e}")
            self.conn.rollback()
    
    def close(self) -> None:
        """
        Note: Connection is managed externally, so we don't close it here.
        """
        pass

### Basic Example: Agent with Persistent Session Memory

These turns demonstrate memory behavior explicitly:
1. Introduce a user identity and topic.
2. Ask follow-up questions that depend on prior context.
3. Remove selected items to test forgetting behavior.
4. Clear the session and verify memory reset.

This gives a practical template for evaluating memory quality in your own assistants.


In [ ]:
# Create an agent
research_agent = Agent(
    name="Assistant",
    instructions="Research the topic and return the most relevant information.",
)

In [ ]:
# Create an Oracle session instance
session = OracleSession(
    session_id="conversation_123", 
    connection=conn,
    table_name="chat_history"
)

In [ ]:
# First turn
result_from_research_agent = await Runner.run(
    starting_agent=research_agent,
    input="Hi my name is Richmond, and I am a AI Memory Engineer researching LLMs and Agent Memory",
    session=session
)

print(f"Assistant: {result_from_research_agent.final_output}")

In [ ]:
# Second turn
result_from_research_agent = await Runner.run(
    starting_agent=research_agent,
    input="What is a paper that introduces the attention mechanism in LLMs?",
    session=session
)

print(f"Assistant: {result_from_research_agent.final_output}")

In [ ]:
# Third turn, the agent will remember the previous conversation
result_from_research_agent = await Runner.run(
    starting_agent=research_agent,
    input="Who were the authors of the paper?",
    session=session
)

print(f"Assistant: {result_from_research_agent.final_output}")

In [ ]:
# Fourth turn - continuing the conversation
result_from_research_agent = await Runner.run(
    starting_agent=research_agent,
    input="What was the year of publication?",
    session=session
)

print(f"Assistant: {result_from_research_agent.final_output}")

In [ ]:
# Change the conversation subject and ensure the agent does't remember the previous conversation
# Specifiying pop without limit will remove the last item in the session
await session.pop_item(limit=7)

In [ ]:
# Fifth turn: The agent should not remember the conversations about the paper
result_from_research_agent = await Runner.run(
    starting_agent=research_agent,
    input="What paper have we been talking about?",
    session=session
)

print(f"Assistant: {result_from_research_agent.final_output}")

In [ ]:
# Because we limited the session to a few items, the agent should still remember our name at the introduction
result_from_research_agent = await Runner.run(
    starting_agent=research_agent,
    input="Do you still remember my name?",
    session=session
)

print(f"Assistant: {result_from_research_agent.final_output}")

In [ ]:
# Clear the session
await session.clear_session()

In [ ]:
# Because we limited the session to 3 items, the agent should still remember our name at the introduction
result_from_research_agent = await Runner.run(
    starting_agent=research_agent,
    input="Do you still remember my name?",
    session=session
)

print(f"Assistant: {result_from_research_agent.final_output}")